In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
import re
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import pandas as pd
data = pd.read_csv('/content/task_1_information_extraction_train.tsv', sep = '\t')
print(len(data))

100000


In [ ]:
data=data[:10000]

In [ ]:
# Removies Null rows and reseting Index
data = data[data['NOTES'].notna()]
data = data.reset_index(drop=True)
# data = data[:20000]

In [ ]:
newdata = pd.concat([data['NOTES'], data['INTER1']], axis=1)

In [ ]:
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [ ]:
counts = Counter()
docs = {}
for i in range(len(newdata['NOTES'])):
    docs[i] = tokenize(newdata['NOTES'][i])
    counts.update(tokenize(newdata['NOTES'][i]))

In [ ]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
newdata['encoded'] = newdata['NOTES'].apply(lambda x: np.array(encode_sentence(x,vocab2index)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
X_train = []
for i in range(len(newdata['encoded'])):
    X_train.append(newdata['encoded'][i][0])

In [ ]:
target = []
for i in range(len(newdata['INTER1'])):
    target.append(newdata['INTER1'][i])

In [ ]:
X_train = np.array(X_train)
print(X_train.shape)
target = np.array(target)

(9696, 70)


In [ ]:

# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(X_train)

X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Inter1
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train1, target, epochs = 500, batch_size = 32)

In [ ]:
testdata = pd.read_csv('/content/task_1_information_extraction_valid.tsv', sep = '\t')
# testdata = testdata

# testdata['NOTES'] = testdata['NOTES'].fillna('')
testdata = testdata[testdata['NOTES'].notna()]
testdata = testdata.reset_index(drop=True)
testdata['encoded'] = testdata['NOTES'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  import sys


In [ ]:
y = []
for i in range(len(testdata['INTER1'])):
    y.append(testdata['INTER1'][i])

y = np.array(y)

In [ ]:
X_test = []
for i in range(len(testdata['encoded'])):
    X_test.append(testdata['encoded'][i][0])
X_test = np.array(X_test)

In [ ]:
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
predicted = regressor.predict(X_test1)

In [ ]:
predictions = []
for i in predicted:
  predictions.append(i.item())

In [ ]:
predictions[:10]

[3.3377845287323,
 4.8231120109558105,
 3.646108627319336,
 3.5548553466796875,
 3.4154322147369385,
 3.39959454536438,
 3.6519358158111572,
 3.4529504776000977,
 3.2848219871520996,
 3.4089789390563965]

In [ ]:
import numpy
corr_matrix = numpy.corrcoef(y, predictions)
corr = corr_matrix[0,1]
R_sq = corr**2*2
 
print("R2 Value for (5, 1)", R_sq)

R2 Value for (5, 1) 0.1791379015126493
